#Informe Final de la Prueba Técnica 2

####**Autor:** Daniela Zabaleta Benitez
####**Fecha:** 20/08/2025
####**Descripción:** Notebook para presentar el informe detallado sobre la prueba técnica #2, se incluye la implementación de un ChatBot Legal utilizando un modelo gratuito de Hugging Face (FLAN-T5), que permite resultados similares y accesibles → Finalmente, FLAN-T5 no me ofreció resultados buenos (devolvia el mismo prompt), entonces terminé implementando el modelo de Gemini (Google Generative AI).


#1. Explicación del caso

####En un consultorio legal quieren automatizar el proceso de consulta y resolución de dudas y optimizar el tiempo de los abogados para asesorar a sus clientes frente a posibles demandas y los resultados de estas, las dudas muy puntuales no pueden ser resueltas solo con la historia de demandas y sentencias, por lo que buscan una solución donde se haga uso de la Inteligencia Artificial Generativa (Generative AI).  
####Para la prueba, quieren concentrarse en demandas relacionadas a redes sociales.

####La solución debe ser capaz de responder preguntas específicas sobre casos legales, en un lenguaje coloquial, es decir, para una persona que no tenga conocimientos de derecho:

*   ¿Cuáles son las sentencias de 3 demandas? (las 3 demandas pueden ser elegidas por ud).
*    ¿De qué se trataron las 3 demandas anteriores?.
*   ¿Cuál fue la sentencia del caso que habla de acoso escolar?.
*   Diga el detalle de la demanda relacionada con acoso escolar.
*   ¿Existen casos que hablan sobre el PIAR? Indique de que trataron los casos y cuáles fueron sus sentencias.







#2. Supuestos

####Se asume que el archivo de datos de casos legales entregado contiene la información necesaria para responder las preguntas planteadas, entonces no es necesario buscar en fuentes externas.
####Tambien como no tengo conocimientos sobre terminologia juridica, se asume que el contenido es correcto, por lo que esta completo y sin errores graves para ser usado.

#3. Formas de resolver el caso y la opcion elegida

####La mejor manera de darle respuesta a las dudas muy puntuales de los clientes que no tienen los conocimientos tecnicos sobre las sentencias legales es creando un ChatBot, que permita realizar preguntas en lenguaje natural y recibir respuestas relevantes basadas en los datos del archivo proporcionado.
####Se concideraron varias opciones, entre ellas:
*   Modelos OpenAI (GPT-3.5) es la opción más natural y fluida, pero fue descartada porque para que la API Key funcione requiere tener credito en la cuenta.
*   Modelos ligeros en Hugging Face (FLAN-T5) es rapido, corre en CPU, pero presenta limitaciones en español y tienden a repetir el prompt sin dar explicaciones claras, por lo que es necesario un prompt mas guiado.

*   Modelos Open Source más robustos (LLaMA-2, Mistral) ofrecen mejores respuestas en español y mayor capacidad de redacción, solo que consumen más recursos, y el tiempo de ejecución puede ser lento en CPU, requieren GPU/tokens.

*   Gemini (Google Generative AI) opción natural y fluida y con API Key de acceso gratuito.

####La opcion elegida para la implementación del ChatBot legal es usando LLaMa-2 en CPU (en GPU saltaba error por el espacio) combinado con la busqueda exacta de una palabra clave como primera prueba para ver como se comportaba el modelo. Lo elegi porque es gratuito y Open Source, pero el tiempo de respuesta en CPU es muy lento y no terminó de cargar.
####Por esto, se optó por usar FLAN-T5 como modelo de soporte para la prueba, pero la calidad finalmente no es buena.
####Finalmente, se uso Gemini (Google Generative AI) porque esta me permitió cumplir con lo que se pedia (pude terminar de probar) ya que tuve acceso gratuito con API Key, fue compatible con el español y me dio respuestas más naturales y útiles que FLAN-T5.





#4. Resultados del análisis de los datos y los modelos

###4.1 Librerias

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
!pip install faiss-cpu sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 25.1 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from google.colab import userdata
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [ ]:
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key = GEMINI_API_KEY)

###4.2 Cagar los datos

In [ ]:
#Ruta archivo
%cd /content/drive/MyDrive/Colab Notebooks/Prueba Técnica Cientifico de Datos/Prueba tecnica 2

/content/drive/MyDrive/Colab Notebooks/Prueba Técnica Cientifico de Datos/Prueba tecnica 2


In [ ]:
df = pd.read_excel('Datos/sentencias_pasadas.xlsx')
df.head(5)

,#,Relevancia,Providencia,Tipo,Fecha Sentencia,Tema - subtema,resuelve,sintesis
0,1,966965.0,T-185/22,NaN,2022-05-31,NaN,en nombre del pueblo y por mandato de la Const...,En este caso se formula la acción de tutela en...
1,3,963168.0,T-356/21,NaN,2021-10-15,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,El peticionario considera que los accionantes ...
2,5,956201.0,T-351/22,NaN,2022-10-07,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,"administrando justicia en nombre del Pueblo, y...",El periodista accionante acusa al abogado acci...
3,6,955889.0,T-246/21,NaN,2021-07-29,ACCION DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,Se presenta la acción de tutela en contra de u...
4,7,955787.0,T-245A/22,NaN,2022-07-01,ACCION DE TUTELA-Inexistencia de hecho superad...,en nombre del pueblo y por mandato de la Const...,"El accionante, actuando en representación de s..."


###4.3 Preparación de los datos

In [ ]:
df['texto'] = df[['Providencia', 'Tipo', 'Fecha Sentencia', 'Tema - subtema', 'resuelve', 'sintesis']].astype(str).agg(' '.join, axis=1)
df['texto'].iloc[0]

'T-185/22 nan 2022-05-31 nan en nombre del pueblo y por mandato de la Constitución, RESUELVE PRIMERO.- REVOCAR las sentencias expedidas el 6 de enero de 2021, por el Juzgado Veinte Penal Municipal con Función de Control de Garantías de Bogotá, en primera instancia, y el 12 de febrero de 2021, por el Juzgado Cincuenta Penal del Circuito con Función de Conocimiento de Bogotá, en segunda instancia, mediante las cuales concedieron el amparo de los derechos fundamentales al buen nombre y a la honra de Enrique Vallejo Echeverri, Paula Camila Fonseca y la Clínica Envalle S.A.S. En su lugar, DECLARAR IMPROCEDENTE el amparo invocado por los accionantes, por las razones expuestas en la parte motiva de esta providencia. SEGUNDO.- Por Secretaría General, LÍBRENSE las comunicaciones de que trata el artículo 36 del Decreto 2591 de 1991. En este caso se formula la acción de tutela en contra de un particular, alegando la vulneración de los derechos fundamentales al buen nombre y honra, en virtud de la

####4.3.1 Vectorización

In [ ]:
modelo_embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = modelo_embeddings.encode(df["texto"].tolist(), convert_to_numpy=True)

In [ ]:
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

###4.4 Función de búsqueda básica por palabra clave
#####Podia implementar embeddings y FAISS/Chroma para que la búsqueda semántica fuera más potente, pero preferí empezar por algo sencillo para probar el funcionamiento del modelo (pero no termina de cargar con ningun ajuste tras varias pruebas) → Al final implemente embeddings y FAISS porque pense que asi funcionaria mejor el modelo de FLAN-T5 → Finalmente, se integró perfectamente con el modelo elegido (Gemini).

In [ ]:
def buscar_casos(palabra, top_k=5):
    palabra_embedding = modelo_embeddings.encode([palabra], convert_to_numpy=True)
    distances, indices = index.search(palabra_embedding, top_k)
    resultados = df.iloc[indices[0]]
    return resultados[["Providencia", "texto"]]

###4.5 Creación de ChatBot

In [ ]:
#Configuración modelo Gemini
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
#Función del chatbot
def consulta_chatbot(pregunta, top_k=5):
    pregunta_emb = modelo_embeddings.encode([pregunta])
    distances, indices = index.search(pregunta_emb, top_k)
    contexto = df.iloc[indices[0]][["Providencia", "texto"]].to_string(index=False)

    if contexto.strip() == "":
        return "No encontré información relevante en los casos legales."

    prompt = f"""
    Actúa como un asesor legal que explica y responde de forma sencilla para alguien sin conocimientos jurídicos o técnicos, entonces debes resumir y explicar sin terminologías complejas.

    Casos encontrados:
    {contexto}

    Pregunta:{pregunta}

    Respuesta:
    """
    response = model.generate_content(prompt)
    respuesta_final = response.text.strip()

    return respuesta_final

###4.6 Pruebas del ChatBot

In [ ]:
pregunta = "Elige tres demandas, las demandas las puedes elegir tú. ¿Cuáles son las sentencias de 3 demandas?"
respuesta = consulta_chatbot(pregunta)
print(respuesta)

Elegiré tres demandas de las presentadas para resumir sus sentencias de forma sencilla:


**1. Demanda C-278/24:** Esta demanda cuestionaba la constitucionalidad de la Ley 2300 de 2023, que protege la intimidad de los consumidores al regular cómo y cuándo las empresas pueden contactarlos para cobrar deudas.  La Corte Suprema encontró que la ley *sí* es constitucional (la declaró "exequible") porque, aunque protege derechos importantes, no necesita seguir un procedimiento legal tan estricto como el que se usa para leyes que solo protegen derechos fundamentales.  En resumen, la ley que protege la intimidad de los consumidores al regular las comunicaciones de cobros es válida.


**2. Demanda C-540/23:** Esta demanda desafió partes de una ley tributaria (Ley 2277 de 2022) que establecía diferentes reglas para el pago de impuestos para distintos grupos de profesionales independientes bajo el Régimen Simple de Tributación (RST). La Corte encontró que estas diferencias eran injustas e inconsi

In [ ]:
pregunta = "¿De qué se trataron las 3 demandas anteriores?"
respuesta = consulta_chatbot(pregunta)
print(respuesta)

Las tres demandas presentan situaciones distintas, pero todas involucran alegatos de violación de derechos fundamentales:

**Caso C-540/23:** Esta demanda cuestiona la equidad de un sistema tributario simplificado (RST) en Colombia.  Profesionales independientes se quejaron porque el sistema trataba de manera diferente, e injusta, a dos grupos con actividades similares (servicios profesionales), imponiendo límites de ingresos y tarifas de impuestos más altas a un grupo que al otro sin una razón justificable.  La Corte encontró que esta diferencia era arbitraria y la declaró inconstitucional.

**Caso T-584/23:**  Un individuo demandó a una empresa que le otorgó un préstamo a través de una aplicación móvil, por prácticas de cobro de deudas abusivas. La empresa empleó tácticas intimidatorias como ir a la casa de sus familiares, colocar carteles con su imagen y difundir su información personal. La Corte encontró que la empresa había violado los derechos a la intimidad, la honra y el buen n

In [ ]:
pregunta = "¿Cuál fue la sentencia del caso que habla de acoso escolar? "
respuesta = consulta_chatbot(pregunta)
print(respuesta)

La sentencia que trata sobre acoso escolar es la **T-252/23**.  En este caso, un estudiante fue retirado del colegio debido a acoso escolar ("bullying" o matoneo). Aunque el daño ya estaba hecho (daño consumado), la Corte Constitucional aún decidió pronunciarse sobre la violación de los derechos del estudiante y ordenó a la institución educativa tomar medidas para prevenir y solucionar casos futuros de acoso escolar y ciberacoso.


In [ ]:
pregunta = "Diga el detalle de la demanda relacionada con acoso escolar"
respuesta = consulta_chatbot(pregunta)
print(respuesta)

En el caso T-400/20, un estudiante de 15 años fue suspendido indefinidamente por solicitar y obtener fotos íntimas de una niña de 12 años, ambas estudiantes de la misma institución educativa, usando medios electrónicos.  El padre del estudiante demandó, alegando que la suspensión violaba los derechos de su hijo al debido proceso y a una sanción proporcional.

La Corte Constitucional revisó el caso y confirmó que la institución no había violado los derechos del estudiante, aunque sí criticó el lenguaje usado por la institución al referirse a la acción del estudiante como un "delito".  Sin embargo, la Corte sí ordenó a la institución revisar su manual de convivencia para asegurarse de que las sanciones sean justas y proporcionales, y no dependan completamente de la intervención de las autoridades externas.  También se instó al departamento de educación a realizar talleres sobre respeto a la intimidad, uso responsable de redes sociales y los derechos y deberes de los estudiantes.  En resu

In [ ]:
pregunta = "¿Existen casos que hablan sobre el PIAR?. Indique de que trataron los casos y ¿cuáles fueron sus sentencias?"
respuesta = consulta_chatbot(pregunta)
print(respuesta)

No se menciona explícitamente el acrónimo "PIAR" en ninguno de los casos proporcionados.  Es posible que se trate de una sigla no detallada en los extractos de las providencias. Para poder identificar casos relacionados con PIAR, necesito que me proporcione la información completa de qué significa ese acrónimo.


#4.7 Conclusiones

####Al no poder terminar de probar ninguno de los modelos, puedo establecer que LLaMA-2 resulta muy lenta para este tipo de entornos y FLAN-T5 large en CPU corre de forma confiable pero al final las respuesta que se tuvieron anteriormente no sirven para dar solución al proyecto, ya que termina repitiendo la misma pregunta que se la hecho.
####Finalmente, se logró un chatbot legal que entiende preguntas en lenguaje natural y responde con base a información vectorizada.
####Gemini ofreció respuestas más naturales y completas que FLAN-T5.

#5. Futuros ajustes o mejoras

####Teniendo en cuenta las limitaciones que se tuvieron con los modelos Open Source, se podria implementar un modelo OpenAI, que por otro lado tambien tiene un costo, por lo que se podria pensar en entrenar un modelo propio (si se disponen de los recursos necesarios) u optar por explorar alternativas más economicas.
####Por otro lado, para no depender de palabras clave se podria optar por implementar embeddings + FAISS para búsqueda semántica, ya que permite encontrar coincidencias aunque no sean exactas → Al final si implemente embeddings + FAISS para búsqueda semántica porque pense que FLAN-T5 se comportaria mejor, pero finalmente sigue dando como respuesta el mismo prompt o la pregunta que se le hace.

#6. Apreciaciones y comentarios del caso (opcional)

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
import nbformat
notebook_path = "Informe_Final_PT2.ipynb"
notebook_limpio = "Informe_Final_PT2-F.ipynb"

with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

with open(notebook_limpio, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

FileNotFoundError: [Errno 2] No such file or directory: 'Informe_Final_PT2.ipynb'